In [ ]:
from mol_text_align import MolTextAligner
from model.unimol_simple import SimpleUniMolModel
import argparse 

parser = argparse.ArgumentParser()
SimpleUniMolModel.add_args(parser) 
parser.add_argument("--bert-name", type=str, default="molT5")   # choose the text encoder : sci_bert / molT5
parser.add_argument("--vae", type=str, default="./LDMol/checkpoint_autoencoder.ckpt")  
parser.add_argument("--smiles-encoder-dim", type=int, default=64)
parser.add_argument("--text-encoder-embed-dim", type=int, default=1024)   # sci_bert : 768, molT5 : 1024
parser.add_argument("--temperature", type=float, default=0.1)
parser.add_argument("--embed-dim", type=int, default=512)
args = parser.parse_args([])

MTA = MolTextAligner(args)

In [ ]:
from torch.optim import Adam
from rich.progress import Progress
from data_provider.unimol_dataset import batch_provider

data_path = 'C:/Users/namjo/OneDrive/문서/GitHub/Molecule-Text-Alignment/3d-pubchem.lmdb'
bert_name = args.bert_name

lr = 2e-4; weight_decay = 1e-6; n_epochs = 5000

optimizer = Adam(MTA.parameters(), lr=lr, weight_decay=weight_decay)

with Progress() as progress:
    training_task = progress.add_task("[red]Training...", total=n_epochs)

    for epoch in range(n_epochs):
        smiles_tokens, padded_atom_vec, padded_dist, padded_edge_type, token_idxs, attention_masks = batch_provider(data_path, bert_name)
        optimizer.zero_grad()      
        loss_Auto = MTA(smiles_tokens, padded_atom_vec, padded_dist, padded_edge_type, token_idxs, attention_masks)
        loss_Auto.backward()
        optimizer.step()
        progress.update(training_task, advance=1, description=f"Epoch {epoch}/{n_epochs} - Loss: {loss_Auto.item():.4f}")
